In [1]:
import tensorflow as tf
keras = tf.keras

2023-06-13 03:48:40.053173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 03:48:40.943696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model=keras.models.load_model("/tmp/open_spiel_PVPNet_test_mlp.pb")

2023-06-11 00:30:47.411439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
batch_size=30
environment_shape=(20,20,2)
state_shape=(1,)
E=tf.zeros([batch_size,*environment_shape])
S=tf.zeros([batch_size,*state_shape])
model.predict({"inputs_0":E,"inputs_1":S})

ValueError: in user code:

    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [10]:
model.

In [317]:
n=100
G=tf.cast(tf.random.uniform([n,n]) > 0,dtype=tf.float32)
W=tf.random.uniform([n,n]) * G
x=tf.random.uniform([n,1])
x=tf.stack([x,x])
G=tf.stack([G,G])
W=tf.stack([W,W])



import importlib
import mpg.gnn.gcn as gcn
import mpg.gnn.aggregator as agg
import mpg.gnn.sgcn as sgcn 
importlib.reload(gcn)
importlib.reload(agg)
importlib.reload(sgcn)
WGCN=gcn.WGCN

In [320]:
layer=WGCN(5,loops=False,normalization="unit")
layer2=WGCN(5,loops=False,normalization="mean")
Y=layer((G,W,x))
Z=layer2((G,W,Y))


In [321]:
Y

<tf.Tensor: shape=(2, 100, 5), dtype=float32, numpy=
array([[[ 2.364595 , -5.8917894, -1.6306539,  6.2325687, -6.030709 ],
        [ 2.3944468, -5.966171 , -1.6512402,  6.311252 , -6.106844 ],
        [ 2.3923035, -5.9608297, -1.649762 ,  6.3056026, -6.101377 ],
        [ 2.1069312, -5.2497764, -1.4529657,  5.553422 , -5.373558 ],
        [ 2.2156246, -5.5206046, -1.5279222,  5.839915 , -5.650772 ],
        [ 2.2426457, -5.5879326, -1.5465562,  5.9111366, -5.719687 ],
        [ 2.2155795, -5.520492 , -1.5278909,  5.8397956, -5.6506567],
        [ 2.3688703, -5.902442 , -1.6336021,  6.2438374, -6.0416126],
        [ 2.0590339, -5.130432 , -1.4199352,  5.427175 , -5.2514   ],
        [ 2.1686325, -5.403516 , -1.4955158,  5.7160535, -5.5309224],
        [ 2.5228539, -6.2861185, -1.7397912,  6.649706 , -6.434335 ],
        [ 2.2568812, -5.6234026, -1.5563732,  5.9486585, -5.755994 ],
        [ 2.388234 , -5.95069  , -1.6469556,  6.294876 , -6.090998 ],
        [ 2.3148189, -5.7677636, -1.5

[<tf.Variable 'TAGConv/kernel:0' shape=(1, 5, 2) dtype=float32, numpy=
 array([[[-0.04742214, -0.14084202],
         [-0.41123575,  0.13933949],
         [ 0.72605807,  0.03667752],
         [-1.2124313 ,  0.43771482],
         [-0.48633334,  0.4591117 ]]], dtype=float32)>]

In [163]:
aggregator1=agg.SumAggregation(loops=True,with_weights=False,normalization="unit")
def f(A,x):
    matrices=A
    matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
    degrees=tf.reduce_sum(matrices,axis=-1)
    D=tf.sqrt(tf.linalg.diag(degrees))
    return tf.linalg.matmul(tf.linalg.matmul(tf.linalg.pinv(D),tf.linalg.matmul(matrices,tf.linalg.pinv(D))),x)

data=tf.ones([2,5,1])

In [168]:
f(G,x)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]]], dtype=float32)>

In [169]:
aggregator1((G,x))

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]]], dtype=float32)>

In [170]:
import numpy as np
P=np.zeros([2,5,1])
degree = tf.reduce_sum(G,axis=-1) +1 
for b in range(2):
    
    for i in range(5):
        P[b,i,0] = x[b,i,0] / degree[b,i]
        for j in range(5):
            P[b,i,0] += G[b,i,j]*x[b,j,0] / np.sqrt(degree[b,i] * degree[b,j])
P

array([[[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]]])

In [167]:
matrices=G
matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
degrees=tf.reduce_sum(matrices,axis=-1,keepdims=True)
tf.linalg.matmul(matrices, data / tf.math.sqrt(degrees))/tf.math.sqrt(degrees)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]],

       [[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]]], dtype=float32)>

In [61]:
import gc
gc.collect()

1480

In [130]:
import importlib
import mpg.ml.dataset.reader as reader
import mpg.ml.dataset.generator as generator
import mpg.ml.dataset.utils as utils
import mpg.ml.dataset.transforms as transforms
importlib.reload(reader)
importlib.reload(generator)
importlib.reload(utils)
importlib.reload(transforms)
target_path="/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/targets_dense-worker-35347670-0.json"
#target_dataset="/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/targets_dense.json"
gen=reader.MPGGraphReader("/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/dense",target="all",target_path=target_path,debug=True)

(TensorSpec(shape=(2, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(2, 2, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))


In [134]:
T=transforms.WithStartingPosition(2,0)
next(iter(T(gen)))

2023-06-17 04:54:25.400594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


((<tf.Tensor: shape=(2, 90, 90), dtype=float32, numpy=
  array([[[   -1.,    -1.,    -1., ...,    -1.,    -1.,    -1.],
          [   -1.,    -0.,    -1., ...,    -1.,    -1.,    -1.],
          [   -1.,    -0.,    -1., ...,    -1.,    -0.,    -1.],
          ...,
          [   -1.,    -1.,    -0., ...,    -0.,    -1.,    -1.],
          [   -1.,    -1.,    -1., ...,    -1.,    -0.,    -1.],
          [   -1.,    -1.,    -0., ...,    -1.,    -1.,    -1.]],
  
         [[ -191.,   207.,   954., ...,  -462.,    -2.,    31.],
          [ -924.,    -0.,   811., ...,  -703.,  -695.,  -965.],
          [ -929.,    -0.,  -240., ...,   668.,    -0.,   850.],
          ...,
          [ -752.,   595.,    -0., ...,    -0.,  -312.,   587.],
          [  927.,  -827.,   707., ..., -1000.,    -0.,  -949.],
          [  147.,   278.,    -0., ...,   627.,   954.,  -484.]]],
        dtype=float32)>,
  <tf.Tensor: shape=(), dtype=int32, numpy=2>),
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([7.,

In [129]:
import pandas as pd
df=pd.read_json(target_path)
df.set_index("graph",inplace=True)

In [123]:
x,y,z=next(iter(gen))

2023-06-17 04:49:59.634559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [45]:
df.loc["gnp_uniform_mpg_250_175_151.edgelist.gz"]==

dataset                           /lustre/ssd/ws/razo743f-mpg-dataset
max_strategy        [140, 214, 60, 93, 2, 141, 248, 59, 221, 97, 2...
mean_payoffs_max    [-505.094, -505.094, -505.094, -505.094, -505....
mean_payoffs_min    [-505.094, -505.094, -505.094, -505.094, -505....
min_strategy        [157, 223, 83, 153, 215, 95, 105, 110, 105, 49...
running_time                                                12.980278
status                                                             OK
winners_max         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
winners_min         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: gnp_uniform_mpg_250_175_151.edgelist.gz, dtype: object

In [109]:
y[1,1,:] 

<tf.Tensor: shape=(250,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [94]:
def equal_element_wise(x,y):
    return len(x)==len(y) and np.all(x==y)

mask=(df["max_strategy"].map(lambda x:equal_element_wise(x,t)) ) | (df["min_strategy"].map(lambda x: equal_element_wise(x,t)))
df[mask]

,dataset,max_strategy,mean_payoffs_max,mean_payoffs_min,min_strategy,running_time,status,winners_max,winners_min
graph,,,,,,,,,


In [67]:
L=[157,223,83,153,215,95,105,110,105,49,143,171,248,88,85,37,245,124,197,42,89,45,60,33,2,37,218,97,45,236,122,8,60,184,165,134,211,237,95,128,206,119,138,32,120,78,207,201,11,22,141,22,182,80,35,162,64,96,138,107,214,218,45,159,162,2,221,70,80,45,138,23,22,68,219,60,136,54,227,208,91,144,150,218,145,6,55,124,97,14,136,62,85,238,207,15,78,207,229,70,62,43,197,197,110,153,141,144,41,228,94,211,44,142,100,43,238,144,206,44,19,110,140,230,9,14,222,117,144,248,150,154,146,123,205,58,85,160,236,178,181,146,214,149,201,211,218,132,196,141,73,196,218,207,85,78,163,218,242,61,149,207,44,68,32,2,90,147,183,138,85,147,236,145,183,170,121,179,170,81,159,159,64,184,55,243,69,86,201,148,41,19,181,55,21,214,122,117,85,171,36,243,22,43,186,136,155,9,64,128,229,170,54,141,46,80,229,51,170,21,123,223,92,24,101,81,243,127,90,105,218,153,188,201,248,54,218,175,64,116,223,150,175,97,26,218,228,45,214,144]
y[0,1,:] == L

<tf.Tensor: shape=(250,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  Tr

In [72]:
y[0,1,44:47]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([120., 234., 207.], dtype=float32)>

In [37]:
df[mask]

,dataset,max_strategy,mean_payoffs_max,mean_payoffs_min,min_strategy,running_time,status,winners_max,winners_min
graph,,,,,,,,,
gnp_uniform_mpg_250_25_839.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[173, 161, 132, 66, 159, 39, 97, 140, 206, 124...","[-424.909, -424.909, -424.909, -424.909, -424....","[-424.909, -424.909, -424.909, -424.909, -424....","[154, 210, 83, 150, 23, 125, 123, 212, 42, 148...",7.023631,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
gnp_uniform_mpg_250_50_438.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[156, 62, 111, 187, 242, 134, 77, 41, 57, 68, ...","[-229, -217, -210.5, -540.75, -210.5, -210.5, ...","[-210.5, -540.75, -210.5, -217, -210.5, -210.5...","[89, 82, 188, 219, 49, 214, 99, 41, 57, 189, 1...",6.147154,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
gnp_uniform_mpg_250_175_833.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[245, 247, 180, 92, 24, 164, 140, 38, 137, 242...","[-345, -404.05, -404.05, -345, -404.05, -404.0...","[-404.05, -404.05, -404.05, -404.05, -404.05, ...","[30, 220, 136, 109, 242, 235, 3, 93, 199, 122,...",19.117800,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
gnp_uniform_mpg_250_225_403.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[246, 188, 241, 249, 247, 46, 17, 57, 94, 37, ...","[444.1, 444.1, 444.1, 444.1, 444.1, 444.1, 444...","[444.1, 444.1, 223.5, 444.1, 444.1, 444.1, 444...","[162, 54, 11, 128, 128, 92, 157, 225, 231, 187...",26.976864,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
gnp_uniform_mpg_250_175_336.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[43, 191, 40, 48, 43, 195, 112, 7, 166, 43, 48...","[377, 377, 377, 377, 377, 377, 377, 377, 377, ...","[377, 377, 492.5, 377, 377, 377, 377, 377, 377...","[62, 30, 39, 121, 123, 8, 200, 193, 212, 72, 1...",35.757119,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
gnp_uniform_mpg_250_75_876.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[80, 196, 99, 174, 156, 233, 78, 156, 29, 225,...","[-624, -624, -624, -624, -624, -624, -624, -62...","[-624, -624, -624, -624, -624, -624, -624, -62...","[64, 67, 96, 57, 102, 164, 194, 31, 30, 127, 1...",8.639763,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
gnp_uniform_mpg_250_125_723.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[152, 170, 29, 183, 24, 104, 212, 70, 98, 44, ...","[-432, -663, -663, -539.5, -432, -663, -663, -...","[-432, -663, -663, -663, -663, -663, -663, -43...","[16, 19, 188, 194, 143, 41, 223, 186, 26, 29, ...",16.271652,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
gnp_uniform_mpg_250_75_85.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[159, 49, 178, 230, 196, 79, 84, 128, 6, 9, 17...","[423.5, 659.5, 659.5, 659.5, 659.5, 659.5, 659...","[659.5, 659.5, 659.5, 423.5, 659.5, 659.5, 423...","[151, 113, 219, 15, 156, 81, 229, 128, 177, 67...",11.624201,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
for x in gen.bucket_by_sequence_length(lambda x:x.shape[-1],[20,50,500],[5,5,5,5],pad_to_bucket_boundary=True):
    z=x
    break

True

In [74]:
import mpg.mpgio as mpgio
graph=mpgio.read_mpg("/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/dense/gnp_uniform_mpg_250_175_151.edgelist.gz")
graph.tensor_representation

array([[[   1.,    1.,    1., ...,    1.,    1.,    1.],
        [   1.,    1.,    1., ...,    1.,    1.,    0.],
        [   0.,    1.,    1., ...,    0.,    1.,    1.],
        ...,
        [   0.,    1.,    0., ...,    1.,    0.,    1.],
        [   0.,    1.,    0., ...,    1.,    1.,    1.],
        [   0.,    1.,    1., ...,    1.,    1.,    0.]],

       [[-702.,  718., -280., ..., -663.,  596.,  320.],
        [  94., -271.,  147., ..., -968., -862.,    0.],
        [   0., -824.,  998., ...,    0.,  831., -958.],
        ...,
        [   0.,  141.,    0., ..., -574.,    0.,  953.],
        [   0.,  578.,    0., ...,  -90.,  748.,  496.],
        [   0.,  236.,  196., ..., -195.,  410.,    0.]]], dtype=float32)

In [285]:
tf.linalg.det(tf.random.uniform([n,n]) 

SyntaxError: incomplete input (32705229.py, line 1)

In [1]:
import pandas as pd
df=pd.read_json("/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/targets_dense.json")
df.set_index("graph",inplace=True)


In [468]:
filename="gnp_uniform_mpg_300_30_108.edgelist.gz"

In [516]:
B=np.array(df.loc[filename,["winners_max","winners_min"]].to_list()).T
A=np.array(df.loc[filename,["max_strategy","min_strategy"]].to_list()).T
C=np.stack([B,A],axis=1)

In [523]:
C[:,1,0]

array([281, 116, 176, 194, 297, 287, 195, 148,  75, 262, 248, 158,  77,
       173,  14, 283, 118,   4, 102, 282, 191,  43, 191, 171, 245, 184,
       199, 125, 166,  11, 295,  46,  31, 159,  17, 245, 123, 185, 188,
       178, 154, 175, 185,  86, 287, 276, 175, 118,  52,  69, 163,  98,
       261, 261, 264, 264,  14, 274, 248, 242, 225, 213,  58, 234,  99,
        28, 209,  20,  89,  45, 196, 146,  38, 198, 141, 256,  63,  57,
       250, 262, 146, 228, 133, 136, 111,  20, 237,  66,  70,  65,  61,
       193, 130, 118, 256, 256, 168,  35, 185, 188, 122, 241,  21, 258,
       192, 256,   2, 160, 219,  86, 192, 297, 202, 217, 126, 290,  47,
        46, 135, 115,  67, 280,   8, 170, 143, 225, 186, 261,  85,   0,
        22, 115, 281,  50, 127, 195, 194,  45, 111,  86, 128, 252,  53,
       216, 133,  25, 121, 169, 213, 193, 227, 256,  53, 255,   6, 263,
        69, 163, 161, 147,  34,  65, 175, 147,  95, 197, 112, 252, 131,
       261,  90, 289, 144,  95,  91,  38, 172, 143,  82, 124, 21

In [14]:
df=df.reset_index()

,graph,dataset,max_strategy,mean_payoffs_max,mean_payoffs_min,min_strategy,running_time,status,winners_max,winners_min
0,gnp_uniform_mpg_300_30_108.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[281, 116, 176, 194, 297, 287, 195, 148, 75, 2...","[-344.1, -593.375, -344.1, -344.1, -344.1, -59...","[-344.1, -344.1, -344.1, -344.1, -344.1, -344....","[251, 40, 268, 231, 129, 17, 94, 256, 199, 244...",3.395358,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,gnp_uniform_mpg_300_30_302.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[70, 196, 153, 106, 47, 121, 72, 262, 60, 157,...","[-526.778, -526.778, -526.778, -526.778, -526....","[-526.778, -526.778, -374, -526.778, -526.778,...","[237, 264, 134, 264, 74, 149, 166, 203, 264, 2...",6.332063,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,gnp_uniform_mpg_30_27_226.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[18, 23, 25, 22, 25, 7, 25, 7, 13, 7, 9, 25, 7...","[726, 726, 726, 718, 726, 726, 726, 726, 726, ...","[752.5, 726, 726, 726, 726, 726, 726, 726, 726...","[22, 19, 24, 27, 23, 0, 25, 9, 24, 23, 19, 16,...",0.180240,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,gnp_uniform_mpg_250_75_22.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[33, 73, 46, 103, 140, 70, 6, 141, 173, 237, 1...","[-530.5, -530.5, -530.5, -530.5, -530.5, -530....","[-529.5, -530.5, -530.5, -171.875, -378, -529....","[233, 90, 83, 145, 104, 56, 229, 81, 69, 107, ...",12.761818,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,gnp_uniform_mpg_60_60_865.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[11, 4, 39, 54, 49, 57, 1, 13, 47, 34, 7, 41, ...","[-306.167, -555, -306.167, -555, -555, -306.16...","[-306.167, -555, -555, -555, -555, -306.167, -...","[16, 40, 54, 17, 54, 16, 1, 16, 9, 21, 6, 16, ...",0.820927,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...
159996,gnp_uniform_mpg_70_14_469.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[69, 39, 12, 18, 7, 41, 8, 23, 11, 38, 3, 11, ...","[529, 529, 529, 529, 529, 529, 529, 529, 529, ...","[529, 529, 529, 529, 529, 529, 529, 529, 529, ...","[35, 19, 45, 24, 29, 61, 8, 61, 67, 18, 48, 15...",0.165900,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
159997,gnp_uniform_mpg_20_14_285.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[17, 18, 18, 11, 17, 11, 12, 7, 0, 18, 7, 10, ...","[937, 915, 915, 937, 937, 937, 915, 937, 937, ...","[937, 162, 937, 937, 915, 915, 162, 937, 915, ...","[18, 16, 12, 3, 9, 2, 16, 7, 2, 18, 10, 4, 6, ...",0.092224,OK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
159998,gnp_uniform_mpg_500_100_913.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[1, 134, 295, 304, 406, 394, 14, 276, 168, 92,...","[-589.639, -589.639, -589.639, -589.639, -589....","[-589.639, -589.639, -589.639, -589.639, -589....","[252, 482, 215, 460, 23, 488, 186, 424, 268, 4...",29.659051,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
159999,gnp_uniform_mpg_150_135_379.edgelist.gz,/lustre/ssd/ws/razo743f-mpg-dataset,"[86, 81, 128, 15, 79, 107, 37, 17, 123, 16, 94...","[-910, -451.333, -451.333, -451.333, -451.333,...","[-910, -910, -910, -910, -451.333, -451.333, -...","[79, 99, 128, 128, 28, 43, 62, 74, 41, 12, 69,...",7.047188,OK,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [18]:
df.set_index(["dataset","graph"],inplace=True)


In [31]:
for _,u in df.iterrows():
    print(u.name)
    print(_)
    break

('/lustre/ssd/ws/razo743f-mpg-dataset', 'gnp_uniform_mpg_300_30_108.edgelist.gz')
('/lustre/ssd/ws/razo743f-mpg-dataset', 'gnp_uniform_mpg_300_30_108.edgelist.gz')


In [34]:
"name" in u.index

False

In [40]:
if "name":
    s=4
else:
    s=3
print(s)

4


[[281,
  116,
  176,
  194,
  297,
  287,
  195,
  148,
  75,
  262,
  248,
  158,
  77,
  173,
  14,
  283,
  118,
  4,
  102,
  282,
  191,
  43,
  191,
  171,
  245,
  184,
  199,
  125,
  166,
  11,
  295,
  46,
  31,
  159,
  17,
  245,
  123,
  185,
  188,
  178,
  154,
  175,
  185,
  86,
  287,
  276,
  175,
  118,
  52,
  69,
  163,
  98,
  261,
  261,
  264,
  264,
  14,
  274,
  248,
  242,
  225,
  213,
  58,
  234,
  99,
  28,
  209,
  20,
  89,
  45,
  196,
  146,
  38,
  198,
  141,
  256,
  63,
  57,
  250,
  262,
  146,
  228,
  133,
  136,
  111,
  20,
  237,
  66,
  70,
  65,
  61,
  193,
  130,
  118,
  256,
  256,
  168,
  35,
  185,
  188,
  122,
  241,
  21,
  258,
  192,
  256,
  2,
  160,
  219,
  86,
  192,
  297,
  202,
  217,
  126,
  290,
  47,
  46,
  135,
  115,
  67,
  280,
  8,
  170,
  143,
  225,
  186,
  261,
  85,
  0,
  22,
  115,
  281,
  50,
  127,
  195,
  194,
  45,
  111,
  86,
  128,
  252,
  53,
  216,
  133,
  25,
  121,
  169,
  213,
  193